# ECJ Beta Test: Data Summary and Exploration
A look at the data we gathered as part of the ECJ Beta test.

In [1]:
import sys
sys.path.append("../")
from src.data import make_airthings_dataset

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

<a id="toc"></a>

# Table of Contents
1. [Experiment Details](#details)
2. [Full Beta Test](#beta)
3. [AirThings and Beacon Integration](#airthings_beacon_integration)

---

<a id="details"></a>

# Experiment Details
Important Dates:
* Deployment Start: 02/10/2022
* RPi Beacon Deployment Start (3rd Floor): 03/01/2022
* Deployment End (5th Floor): 03/10/2022
* Deployment End (3rd Floor): 03/29/2022

We have two separate sub-tests we want to consider:
1. [Full Beta Test](#beta): Data should be available from all _ten_ devices from 02/10 to 03/10
2. [Third Floor AirThings and Beacon Integration Test](#airthings_beacon_integration): Data should be available from _five_ devices from 03/01 to 03/29

---

<a id="beta"></a>

[Back to ToC](#toc)
# Beta Test

---

<a id="airthings_beacon_integration"></a>

[Back to ToC](#toc)
# AirThings and Beacon Integration Test
I developed a script that pulls data from each of the RPi Beacons based on their Tailscale IP address. See the [source code](https://github.com/intelligent-environments-lab/bleed-orange-measure-iaq/blob/master/src/data/make_airthings_dataset.py).

The data imported below is the output from running that script.

In [4]:
at_data = pd.read_csv("../data/processed/airthings-data.csv",
                      index_col="timestamp",parse_dates=True,infer_datetime_format=True)

In [5]:
at_data["device"].unique()

array([2930041730, 2930041961, 2930042307, 2930042428, 2930042521])